In [ ]:
!pip install albumentations > /dev/null
!git clone https://github.com/qubvel/efficientnet.git
!pip install image-classifiers==0.2.2 

In [ ]:
import os
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np
import keras
import pandas as pd
from PIL import Image
import cv2

from keras.applications import Xception,ResNet50
from keras.applications.imagenet_utils import decode_predictions

from efficientnet import EfficientNetB0,EfficientNetB3
from classification_models.resnet import ResNet101,preprocess_input


# from efficientnet import center_crop_and_resize, preprocess_input
from keras.optimizers import SGD, Adam

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels,augmentations, batch_size=32, dim=(32,32,32), n_channels=3,
                 n_classes=5, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.path = '../input/game-of-deep-learning-ship-datasets/train/images/'
        self.augment = augmentations
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.data_generation(list_IDs_temp)

        return np.stack([
            self.augment(image=x)["image"] for x in X
        ], axis=0), np.array(y)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
#         print(X.shape,self.dim)
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            
            im = np.array(Image.open(self.path+ID))
            if len(im.shape)==2:
                im = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)

#             # Resize sample
            X[i,] = cv2.resize(im,(self.dim[0],self.dim[1]))

            # Store class
            y[i] = self.labels.loc[ID].category

#         print(X.shape)
        return np.uint8(X), keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
import cv2
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,OneOf,
    ToFloat, ShiftScaleRotate,GridDistortion, ElasticTransform, JpegCompression, HueSaturationValue,
    RGBShift, RandomBrightness, RandomContrast, Blur, MotionBlur, MedianBlur, GaussNoise,CenterCrop,
    IAAAdditiveGaussianNoise,GaussNoise,Cutout
)

AUGMENTATIONS_TRAIN = Compose([
    HorizontalFlip(p=0.5),
    RandomContrast(limit=0.2, p=0.5),
    RandomGamma(gamma_limit=(80, 120), p=0.5),
    RandomBrightness(limit=1.2, p=0.5),
    HueSaturationValue(hue_shift_limit=5, sat_shift_limit=20,
                       val_shift_limit=10, p=.5),
#     CenterCrop(height=128, width=128, p=0.5),
    Cutout(p=0.5),
    OneOf([
            MotionBlur(p=0.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.3),
    OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
    # CLAHE(p=1.0, clip_limit=2.0),
    ShiftScaleRotate(
        shift_limit=0.0625, scale_limit=0.1, 
        rotate_limit=15, border_mode=cv2.BORDER_REFLECT_101, p=0.5), 
   # ToFloat(max_value=1)
],p=1)


AUGMENTATIONS_TEST = Compose([
    # CLAHE(p=1.0, clip_limit=2.0),
  #  ToFloat(max_value=1)
],p=1)

In [ ]:
class TestDataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs,augmentations, batch_size=32, dim=(32,32,32), n_channels=3,
                 n_classes=5, shuffle=False,flip=False,path=None):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        if path is not None:
            self.path = path
        else:
            self.path = '../input/avgameofdltestjpg/test-jpg/test-jpg/'
        self.on_epoch_end()
        self.augment = augmentations
        self.flip = flip
#         print(len(self.list_IDs))

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
#         if  im_left < self.batch_size:
#             print('*','index',index,len(self.list_IDs) - (index)*self.batch_size )
#             indexes = self.indexes[(index)*self.batch_size:]            
#         elif im_left<0: return 
#         else:
#             indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        indexes = self.indexes[index*self.batch_size:min((index+1)*self.batch_size,len(self.list_IDs))]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X = self.data_generation(list_IDs_temp)
        
        return np.stack([
                    self.augment(image=x)["image"] for x in X
                ], axis=0)
#         return X

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))

    def data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((len(list_IDs_temp), *self.dim, self.n_channels))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            
            im = np.array(Image.open(self.path+ID))
            if len(im.shape)==2:
                im = cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)

            # Resize sample
            if self.flip:
                X[i,] = np.fliplr(cv2.resize(im,(self.dim[0],self.dim[1])))
            else:
                X[i,] = cv2.resize(im,(self.dim[0],self.dim[1]))


        return np.uint8(X)

In [ ]:
train = pd.read_csv('../input/game-of-deep-learning-ship-datasets/train/train.csv')
train['category'] = train['category'] - 1

In [ ]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Maximum
from keras.layers import ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical

from sklearn.model_selection import StratifiedKFold
from skimage.transform import resize as imresize
from tqdm import tqdm


In [ ]:
def plot_loss_acc(history):
    plt.figure(figsize=(20,7))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'][1:])    
    plt.plot(history.history['val_loss'][1:])    
    plt.title('model loss')    
    plt.ylabel('val_loss')    
    plt.xlabel('epoch')    
    plt.legend(['Train','Validation'], loc='upper left')
    
    plt.subplot(1,2,2)
    plt.plot(history.history['acc'][1:])
    plt.plot(history.history['val_acc'][1:])
    plt.title('Model Accuracy')
    plt.ylabel('val_acc')
    plt.xlabel('epoch')
    plt.legend(['Train','Validation'], loc='upper left')
    plt.show()

In [ ]:
test = pd.read_csv('../input/game-of-deep-learning-ship-datasets/test_ApKoW4T.csv')

In [ ]:
from keras import backend as K
import tensorflow as tf
'''
Compatible with tensorflow backend
'''
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [ ]:
class SnapshotCallbackBuilder:
    def __init__(self, nb_epochs, nb_snapshots, init_lr=0.1):
        self.T = nb_epochs
        self.M = nb_snapshots
        self.alpha_zero = init_lr

    def get_callbacks(self, model_prefix='Model'):

        callback_list = [
#             callbacks.ModelCheckpoint("./keras.model",monitor='val_loss', 
#                                    mode = 'min', save_best_only=True, verbose=1),
            swa,
            callbacks.LearningRateScheduler(schedule=self._cosine_anneal_schedule)
        ]

        return callback_list

    def _cosine_anneal_schedule(self, t):
        cos_inner = np.pi * (t % (self.T // self.M))  # t - 1 is used when t has 1-based indexing.
        cos_inner /= self.T // self.M
        cos_out = np.cos(cos_inner) + 1
        return float(self.alpha_zero / 2 * cos_out)

import keras.callbacks as callbacks

class SWA(keras.callbacks.Callback):
    
    def __init__(self, filepath, swa_epoch):
        super(SWA, self).__init__()
        self.filepath = filepath
        self.swa_epoch = swa_epoch 
    
    def on_train_begin(self, logs=None):
        self.nb_epoch = self.params['epochs']
        print('Stochastic weight averaging selected for last {} epochs.'
              .format(self.nb_epoch - self.swa_epoch))
        
    def on_epoch_end(self, epoch, logs=None):
        
        if epoch == self.swa_epoch:
            self.swa_weights = self.model.get_weights()
            
        elif epoch > self.swa_epoch:    
            for i in range(len(self.swa_weights)):
                self.swa_weights[i] = (self.swa_weights[i] * 
                    (epoch - self.swa_epoch) + self.model.get_weights()[i])/((epoch - self.swa_epoch)  + 1)  

        else:
            pass
        
    def on_train_end(self, logs=None):
        self.model.set_weights(self.swa_weights)
        print('Final model parameters set to stochastic weight average.')
        self.model.save_weights(self.filepath)
        print('Final stochastic averaged weights saved to file.')

In [ ]:
from keras.layers.core import Layer
from keras.engine import InputSpec
from keras import backend as K
try:
    from keras import initializations
except ImportError:
    from keras import initializers as initializations

class Scale(Layer):
    '''Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:
        out = in * gamma + beta,
    where 'gamma' and 'beta' are the weights and biases larned.
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).
        momentum: momentum in the computation of the
            exponential average of the mean and standard deviation
            of the data, for feature-wise normalization.
        weights: Initialization weights.
            List of 2 Numpy arrays, with shapes:
            `[(input_shape,), (input_shape,)]`
        beta_init: name of initialization function for shift parameter
            (see [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
    '''
    def __init__(self, weights=None, axis=-1, momentum = 0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializations.get(beta_init)
        self.gamma_init = initializations.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        # Compatibility with TensorFlow >= 1.0.0
        self.gamma = K.variable(self.gamma_init(shape), name='{}_gamma'.format(self.name))
        self.beta = K.variable(self.beta_init(shape), name='{}_beta'.format(self.name))
        #self.gamma = self.gamma_init(shape, name='{}_gamma'.format(self.name))
        #self.beta = self.beta_init(shape, name='{}_beta'.format(self.name))
        self.trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
from keras import backend as K
from keras.callbacks import  EarlyStopping, Callback
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import  Input, Conv2D, add , MaxPooling2D,Activation,Dropout,Flatten,Dense,BatchNormalization,AveragePooling2D,ZeroPadding2D,merge,Reshape
from keras.optimizers import Adam, SGD
import h5py
import sys
sys.setrecursionlimit(3000)

### Image Flow Generator

class resnet101:
    def __init__ (self, width,height,channel,num_classes):
        self.width = width
        self.height = height
        self.channel = channel
        self.eps = 1.1e-5
        self.num_classes = num_classes
        self.bn_axis = 3

    def identity_block(self, input_tensor, kernel_size, filters, stage, block,strides=(2,2)):
        '''
        Identity Block : This block create convolutional layers that has no convolutional layer at shortcut, where shortcut is used if __name__ == '__main__':
            refrence to the paper describing Resnet Models : https://arxiv.org/pdf/1512.03385.pdf .
        #Args:
            input_tensor: input tensor
            kernel_size : (default value is 3 ) , the kernel size of middle convolutional layer in the Block
            filters : lists of integer , Number of filters
            stage : integer , current state label , used for generating layer names
            block: current block label , used for generating layer names
       #Output : a tensor flow
         '''

        nb_filter1, nb_filter2, nb_filter3 = filters

        conv_name_base = 'res101' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'
        scale_name_base = 'scale' + str(stage) + block + '_branch'

        t = Conv2D(nb_filter1,(1,1), name=conv_name_base + '2a', use_bias = False)(input_tensor)
        t = BatchNormalization(epsilon=self.eps, axis = self.bn_axis, name= bn_name_base + '2a' )(t)
        t = Scale(axis = self.bn_axis, name=scale_name_base + '2a')(t)
        t = Activation('relu', name=conv_name_base + '2a_relu' )(t)

        t = ZeroPadding2D((1,1),name=conv_name_base + '2b_zeropadding')(t)
        t = Conv2D(nb_filter2,(kernel_size,kernel_size), use_bias=False,name=conv_name_base + '2b' )(t)
        t = BatchNormalization(epsilon=self.eps, axis= self.bn_axis, name=bn_name_base + '2b')(t)
        t = Scale(axis = self.bn_axis, name=scale_name_base + '2b')(t)
        t = Activation('relu', name=conv_name_base + '2b_relu')(t)

        t = Conv2D(nb_filter3,(1,1), use_bias=False,name=conv_name_base + '2c' )(t)
        t = BatchNormalization(epsilon=self.eps, axis= self.bn_axis, name=bn_name_base + '2c')(t)
        t = Scale(axis = self.bn_axis, name=scale_name_base + '2c')(t)

        t = add([t,input_tensor])
        t = Activation('relu', name='res' + str(stage) + block +'_relu')(t)
        return t

    def conv_block(self, input_tensor, kernel_size, filters, stage, block,strides=(2,2)):
        '''
        Conv Block : This block create convolutional layers that has a convolutional layer at shortcut, where shortcut is used if __name__ == '__main__':
            refrence to the paper describing Resnet Models : https://arxiv.org/pdf/1512.03385.pdf .
        #Args:
            input_tensor: input tensor
            kernel_size : (default value is 3 ) , the kernel size of middle convolutional layer in the Block
            filters : lists of integer , Number of filters
            stage : integer , current state label , used for generating layer names
            block: current block label , used for generating layer names
       #Output : a tensor flow
         '''

        nb_filter1, nb_filter2, nb_filter3 = filters

        conv_name_base = 'res101' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'
        scale_name_base = 'scale' + str(stage) + block + '_branch'

        t = Conv2D(nb_filter1,(1,1), strides= strides , use_bias=False, name=conv_name_base + '2a')(input_tensor)
        t = BatchNormalization(epsilon=self.eps, axis = self.bn_axis, name= bn_name_base + '2a' )(t)
        t = Scale(axis = self.bn_axis, name=scale_name_base + '2a')(t)
        t = Activation('relu', name=conv_name_base + '2a_relu' )(t)

        t = ZeroPadding2D((1,1),name=conv_name_base + '2b_zeropadding')(t)
        t = Conv2D(nb_filter2,(kernel_size,kernel_size), use_bias=False,name=conv_name_base + '2b' )(t)
        t = BatchNormalization(epsilon=self.eps, axis= self.bn_axis, name=bn_name_base + '2b')(t)
        t = Scale(axis = self.bn_axis, name=scale_name_base + '2b')(t)
        t = Activation('relu', name=conv_name_base + '2b_relu')(t)

        t = Conv2D(nb_filter3,(1,1), use_bias=False,name=conv_name_base + '2c' )(t)
        t = BatchNormalization(epsilon=self.eps, axis= self.bn_axis, name=bn_name_base + '2c')(t)
        t = Scale(axis = self.bn_axis, name=scale_name_base + '2c')(t)

        shortcut = Conv2D(nb_filter3, (1,1), strides=strides,use_bias=False,  name=conv_name_base + '1' )(input_tensor)
        shortcut = BatchNormalization(epsilon=self.eps, axis= self.bn_axis, name=bn_name_base + '1')(shortcut)
        shortcut = Scale(axis=self.bn_axis,  name=scale_name_base + '1')(shortcut)

        t = add([t,shortcut])
        t = Activation('relu', name='res' + str(stage) + block +'_relu')(t)
        return t


    def ResNet101(self,color_type= 1, num_classes = None):
        '''
            ResNet101_v2 Model for Keras
            436 are the Number of layers of model() that were frozen . From  Input layer (conv1) to conv4_x are frozen and conv5_x and
            average pool are trained on input image data .
        '''
        img_input = Input(shape=(self.width,self.height, 3 ), name='data')

        t = ZeroPadding2D((3,3), name='conv1_zeropadding')(img_input)
        t = Conv2D(64, (7,7), strides = (2,2) , use_bias = False , name='conv1' )(t)
        t = BatchNormalization(epsilon = self.eps, axis= self.bn_axis, name='bn_conv1')(t)
        t = Scale(axis = self.bn_axis, name='scale_conv1')(t)
        t = Activation('relu', name= 'conv1_relu')(t)
        t = MaxPooling2D((3,3),strides = (2,2) , name= 'pool1')(t)

        t = self.conv_block(t, 3, [64,64,256], stage=2 , block= 'a' , strides=(1,1))
        t = self.identity_block(t,3,[64,64,256], stage=2, block='b')
        t = self.identity_block(t,3,[64,64,256], stage=2, block='c')

        t = self.conv_block(t, 3, [128,128,512], stage=3 , block= 'a' )
        for i in range(1,4):
            t = self.identity_block(t,3,[128,128,512], stage=3 , block = 'b'+str(i))

        t = self.conv_block(t,3,[256,256,1024], stage=4, block='a')
        for i in range(1,23):
            t = self.identity_block(t,3,[256,256,1024], stage = 4 , block='b' + str(i))

        t = self.conv_block(t,3,[512,512,2048], stage= 5, block= 'a')
        t = self.identity_block(t,3, [512,512,2048], stage=5, block= 'b')
        t = self.identity_block(t,3, [512,512,2048], stage=5, block= 'c')

        x_fc = AveragePooling2D((7,7), name= 'avg_pool')(t)
        x_fc = Flatten()(x_fc)
        x_fc = Dense(1000, activation='softmax', name='fc1000')(x_fc)


        #Pretrained weights
#         weights_path = '../resnet101_weights_tf.h5'


        # Truncate and replace softmax layer for transfer learning
        # The method below works since pre-trained weights are stored in layers but not in the model

        x_newfc = AveragePooling2D((7,7), name='avg_pool')(t)
        x_newfc = Flatten()(x_newfc)
        x_newfc = Dense(self.num_classes, activation='softmax', name= 'fc8')(x_newfc)

        model = Model(img_input, x_newfc)

#         model.load_weights(weights_path, by_name= True)

        for i in range(436):
            model.layers[i].trainable = False

        sgd = SGD(lr= 1e-3, decay=1e-6, momentum=0.9, nesterov= True)
        model.compile(optimizer=sgd , loss='categorical_crossentropy', metrics=['accuracy'])

        return model

In [ ]:
from keras.applications import resnet50

In [ ]:
from keras.layers import Dense, Activation, Flatten, Dropout,GlobalAveragePooling2D
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
#     for layer in base_model.layers:
#         layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
#     print(x.shape)
#     x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

In [ ]:
skf = StratifiedKFold(5)
all_preds,fold = [],1

HEIGHT = 128
WIDTH = 128

input_shape=(HEIGHT, WIDTH, 3)

FC_LAYERS = [2048]
dropout = 0.25
epochs = 60
swa = SWA('./keras_swa.model',epochs-3)

oof_preds = np.zeros((len(train), 6))


# Parameters
params = {'dim': (256,256),
          'batch_size': 32,
          'n_classes': 5,
          'n_channels': 3,
          'shuffle': True}

for train_indices,val_indices in skf.split(train.image.values,train.category.values):
    
    print('*'*50)
    print('Fold',fold)
    fold += 1
    
#     # Datasets
    partition_train = train.loc[train_indices].image.values
    labels_train = train.loc[train_indices].set_index('image')
    
    partition_valid = train.loc[val_indices].image.values
    labels_valid = train.loc[val_indices].set_index('image')

#     # Generators
    training_generator = DataGenerator(partition_train, labels_train,augmentations=AUGMENTATIONS_TRAIN, **params)
    validation_generator = DataGenerator(partition_valid, labels_valid,augmentations=AUGMENTATIONS_TEST, **params)
    
#     base_model = ResNet101(weights='imagenet',
#                                 include_top=False,
#                                 input_shape=(HEIGHT, WIDTH, 3))

#     res101 = resnet101(256,256,3,5)
#     finetune_model = res101.ResNet101(3,5)
    base_model = resnet50.ResNet50(include_top=False,input_shape=(256,256,3))


    finetune_model = build_finetune_model(base_model, 
                                          dropout=dropout, 
                                          fc_layers=FC_LAYERS, 
                                          num_classes=5)
    
    finetune_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
#     for layer in finetune_model.layers[:-3]:
#         layer.trainable = False
        
#     for layer in finetune_model.layers[-3:]:
#         layer.trainable = True
    
    snapshot = SnapshotCallbackBuilder(nb_epochs=epochs,nb_snapshots=1,init_lr=1e-3)


    history = finetune_model.fit_generator(generator=training_generator,
                                            validation_data=validation_generator,
                                            use_multiprocessing=True,
                                            workers=8,epochs=epochs,verbose=0,callbacks=snapshot.get_callbacks())
    
    finetune_model.load_weights('./keras_swa.model')
    
    try:
        plot_loss_acc(history)
    except:
        print('no plot')
    
    test_generator = TestDataGenerator(test.image.values,augmentations=AUGMENTATIONS_TEST, **params)
    test_generator_flipped = TestDataGenerator(test.image.values,augmentations=AUGMENTATIONS_TEST, **params,
                                               flip=True)
    
    
    validation_generator = TestDataGenerator(partition_valid,augmentations=AUGMENTATIONS_TEST, **params,
                                            path='../input/game-of-deep-learning-ship-datasets/train/images/')
    validation_generator_flipped = TestDataGenerator(partition_valid,augmentations=AUGMENTATIONS_TEST,
                                             **params,flip=True,
                                                path='../input/game-of-deep-learning-ship-datasets/train/images/')
    preds1 = []
    for im in validation_generator:    
        preds1.extend(finetune_model.predict(im))
        
    preds2 = []
    for im in validation_generator_flipped:    
        preds2.extend(finetune_model.predict(im))
        
    preds1 = np.array(preds1)
    preds2 = np.array(preds2)
    
    preds = (preds1 + preds2)/2
    
    oof_preds[val_indices, :5] = preds
    oof_preds[val_indices, 5] = train.loc[val_indices,'image'].map(lambda x: x[:-4])
    
    preds1 = []
    for im in test_generator:    
        preds1.extend(finetune_model.predict(im))
        
    preds2 = []
    for im in test_generator_flipped:    
        preds2.extend(finetune_model.predict(im))
        
    preds1 = np.array(preds1)
    preds2 = np.array(preds2)
    
    preds = (preds1 + preds2)/2
    
    pd.DataFrame(preds).to_csv(f'raw_preds_{fold}.csv',index=False)
        
    all_preds.append(preds)

In [ ]:
# pd.DataFrame(preds).to_csv('raw_preds_fold5.csv',index=False)

In [ ]:
pd.DataFrame(oof_preds).to_csv('oof_preds.csv',index=False)

In [ ]:
preds = np.ones((2680,5))
for preds_fold in all_preds:
    preds = preds*preds_fold


In [ ]:
preds = preds**(1/skf.n_splits)
test = pd.read_csv('../input/game-of-deep-learning-ship-datasets/test_ApKoW4T.csv')
labelled_preds = np.argmax(preds,1)+1
# fnames = [f.name for f in learn.data.test_ds.items]
fnames = test.image.values
df = pd.DataFrame({'image':fnames, 'category':labelled_preds}, columns=['image', 'category'])
df.to_csv('submission_gm.csv', index=False)
df.head()

In [ ]:
df.tail()

In [ ]:
df['class1_prob'] = preds[:,0].numpy()
df['class2_prob'] = preds[:,1].numpy()
df['class3_prob'] = preds[:,2].numpy()
df['class4_prob'] = preds[:,3].numpy()
df['class5_prob'] = preds[:,4].numpy()
df.to_csv('raw_prob.csv', index=False)

In [ ]:
!rm -rf efficientnet/